In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Dropout, merge
from keras.layers import LSTM,Bidirectional,GRU,SimpleRNN
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D, Input

import tensorflow as tf
from keras.layers import Conv1D, GlobalMaxPooling1D,GlobalAveragePooling1D,MaxPooling1D, AveragePooling1D
from scipy.stats import pearsonr
from sklearn.cross_validation import train_test_split

Using Theano backend.


In [2]:
import keras.backend as K

def pearson(X,Y):
    #return K.mean(K.abs(y_true-y_pred))
    if K.sum(X)==0:return 0
    n = K.sum(X)/K.mean(X)
    if n==0:return 0
    num = K.sum(X*Y) - K.sum(X)*K.sum(Y)/n
    den = (K.sum(X*X) - K.square(K.sum(X))/n)*(K.sum(Y*Y) - K.square(K.sum(Y))/n)
    den = K.sqrt(den)
    if den==0 : return 0
    return num/den

def pearson1(X,Y):
    #return K.mean(K.abs(y_true-y_pred))
    if K.sum(X)==0:return 0
    n = K.sum(X)/K.mean(X)
    if n==0:return 0
    num = K.sum(X*Y) - K.sum(X)*K.sum(Y)/n
    den = (K.sum(X*X) - K.square(K.sum(X))/n)*(K.sum(Y*Y) - K.square(K.sum(Y))/n)
    den = K.sqrt(den)
    if den==0 : return 0
    return -1 * num/den

# prepare train, dev, test data

In [3]:
emotions = ['joy']
x_train_1, y_train, x_test_1, y_test, x_train_2, x_test_2 = {},{},{},{},{},{}
for emotion in emotions : 
    
    x1t = np.load('../intermediates/train/'+emotion+'/X_'+emotion+'.npy')
    x2t = np.load('../intermediates/train/'+emotion+'/lexicon.npy')
    x3t = np.load('../intermediates/train/'+emotion+'/train_'+emotion+'.npy')
    yt = np.load('../intermediates/train/'+emotion+'/labels.npy')
    x1d = np.load('../intermediates/val/'+emotion+'/X_'+emotion+'_dev.npy')
    x2d = np.load('../intermediates/val/'+emotion+'/lexicon.npy')
    x3d = np.load('../intermediates/val/'+emotion+'/dev_'+emotion+'.npy')
    yd = np.load('../intermediates/val/'+emotion+'/labels.npy')    
    x1te = np.load('../intermediates/test/'+emotion+'/X_'+emotion+'.npy')
    x2te = np.load('../intermediates/test/'+emotion+'/lexicon.npy')
    x3te = np.load('../intermediates/test/'+emotion+'/test_'+emotion+'.npy')
    yte = np.load('../intermediates/test/'+emotion+'/labels.npy')
    
    x1t = np.concatenate([x1t,x2t], axis=1)
    x1d = np.concatenate([x1d,x2d], axis=1)
    x1te = np.concatenate([x1te,x2te], axis=1)
    
    x1t = np.concatenate([x1t,x1d], axis=0)
    x3t = np.concatenate([x3t,x3d], axis=0)
    yt = np.concatenate([yt,yd], axis=0)
    
    x_train_1[emotion] = x1t
    y_train[emotion] = yt
    x_test_1[emotion] = x1te
    y_test[emotion] = yte
    x_train_2[emotion] = x3t
    x_test_2[emotion] = x3te
    print emotion + ': ', x1t.shape, yt.shape, x1te.shape, yte.shape, x3t.shape, x3te.shape
    

joy:  (902, 443) (902,) (714, 443) (714,) (902, 50, 400) (714, 50, 400)


# ensemble of feed-forward NN and CNN/LSTM for each emotion

## anger

In [4]:
ypred = np.zeros(714)
for i in range(7) : 
    input1 = Input(shape=(443,))
    l1 = Dense(300, init='normal', activation='relu')(input1)
    #l1 = Dropout(p=0.2)(l1)
    l1 = Dense(100, init='normal', activation='relu')(l1)
    l1 = Dense(50, init='normal', activation='relu')(l1)
    #l1 = Dropout(p=0.2)(l1)
    l1 = Dense(25, init='normal', activation='relu')(l1)
    #o1 = Dense(1, init='normal')(l1)

    #cnn-lstm-1
    input2 = Input(shape=(50,400,))
    l2 = Conv1D(256,3,activation='relu')(input2)
    l2 = GlobalMaxPooling1D()(l2)
    #l2 = Dropout(p=0.3)(l2)
    l2 = Dense(100,activation='relu')(l2)
    #l2 = Dense(25,activation='relu')(l2)
    #o2 = Dense(1)(l2)

    merged_output = merge([l1, l2], mode='concat', concat_axis=-1)
    merged_output = Dense(25, activation='relu')(merged_output)
    merged_output = Dropout(p=0.2)(merged_output)
    predictions = Dense(1, activation='sigmoid')(merged_output)

    model = Model(input=[input1, input2], output=predictions)
    model.compile(loss='mae', optimizer='adam', metrics=[pearson])
    model.fit([x_train_1['joy'], x_train_2['joy']], y_train['joy'], 
              nb_epoch=20, verbose=2, batch_size=8,
             )
    tmp = model.predict([x_test_1['joy'], x_test_2['joy']])
    print pearsonr(tmp[:,0], y_test['joy'])
    ypred += tmp[:,0]
print pearsonr(ypred/7.0, y_test['joy'])
np.save('joy1.npy', ypred/7.0)

Epoch 1/20
5s - loss: 0.1509 - pearson: 0.4336
Epoch 2/20
4s - loss: 0.1235 - pearson: 0.6973
Epoch 3/20
4s - loss: 0.1015 - pearson: 0.7796
Epoch 4/20
4s - loss: 0.0897 - pearson: 0.8412
Epoch 5/20
4s - loss: 0.0820 - pearson: 0.8674
Epoch 6/20
4s - loss: 0.0708 - pearson: 0.8993
Epoch 7/20
5s - loss: 0.0667 - pearson: 0.9121
Epoch 8/20
6s - loss: 0.0640 - pearson: 0.9196
Epoch 9/20
6s - loss: 0.0594 - pearson: 0.9317
Epoch 10/20
6s - loss: 0.0583 - pearson: 0.9278
Epoch 11/20
6s - loss: 0.0578 - pearson: 0.9334
Epoch 12/20
6s - loss: 0.0567 - pearson: 0.9306
Epoch 13/20
6s - loss: 0.0509 - pearson: 0.9498
Epoch 14/20
6s - loss: 0.0515 - pearson: 0.9437
Epoch 15/20
6s - loss: 0.0508 - pearson: 0.9485
Epoch 16/20
7s - loss: 0.0487 - pearson: 0.9537
Epoch 17/20
9s - loss: 0.0453 - pearson: 0.9600
Epoch 18/20
8s - loss: 0.0457 - pearson: 0.9605
Epoch 19/20
9s - loss: 0.0441 - pearson: 0.9579
Epoch 20/20
9s - loss: 0.0443 - pearson: 0.9582
(0.72841576735575098, 4.8463926575926278e-119)
Ep

## fear 